In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
raw_df_match_details = pd.read_csv('../data/raw/match_details.csv')
raw_df_player_attr = pd.read_csv('../data/raw/player_attributes.csv')

In [3]:
# raw_df_match_details.info()

In [4]:
# raw_df_player_attr.info()

In [5]:
raw_df_match_details['date'] = pd.to_datetime(raw_df_match_details['date'])
raw_df_player_attr['date'] = pd.to_datetime(raw_df_player_attr['date'])
raw_df_match_details.sort_values(by='date', inplace=True)
raw_df_match_details[['season', 'result_match']] = raw_df_match_details[['season', 'result_match']].astype('category')

In [6]:
from playerstats import player_stats

players_cols = ['{}_player_{}'.format(team, i) for team in ['home', 'away'] for i in range(1, 12)]

player_stats_dict_series = raw_df_match_details.apply(
    lambda row: player_stats.calculate_player_stat_(
        match_row=row,
        df_matches=raw_df_match_details,
        df_player_attr=raw_df_player_attr,
        players=players_cols
    ),
    axis=1
)

new_player_stats_df = pd.json_normalize(player_stats_dict_series)

df = pd.merge(raw_df_match_details, new_player_stats_df, how='left', on='match_api_id')
df.drop(players_cols, axis=1, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3040 entries, 0 to 3039
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   match_api_id                  3040 non-null   int64         
 1   season                        3040 non-null   category      
 2   stage                         3040 non-null   int64         
 3   date                          3040 non-null   datetime64[ns]
 4   away_team                     3040 non-null   int64         
 5   home_team                     3040 non-null   int64         
 6   home_team_goal                3040 non-null   int64         
 7   away_team_goal                3040 non-null   int64         
 8   possession                    3040 non-null   object        
 9   shoton                        3040 non-null   object        
 10  result_match                  3040 non-null   category      
 11  home_possession               